# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin page', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter page', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [10]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 6 relevant links


{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [11]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 23 relevant links


{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'learn page', 'url': 'https://huggingface.co/learn'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'docs page', 'url': 'https://huggingface.co/docs'},
  {'type': 'company page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Discourse forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'Discord community', 'url': 'https://huggingface.co/join/discord'},
  {'ty

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [12]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [13]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
tencent/HunyuanVideo-1.5
Updated
1 day ago
•
1.68k
•
680
facebook/sam3
Updated
5 days ago
•
115k
•
668
black-forest-labs/FLUX.2-dev
Updated
about 8 hours ago
•
59
•
396
Supertone/supertonic
Updated
4 days ago
•
4.11k
•
279
facebook/sam-3d-objects
Updated
5 days ago
•
188
Browse 1M+ models
Spaces
Running
on
Zero
MCP
Featured
1.28k
Qwen Image Edit Camera Control
🎬
1.28k
Fast 4 step inference with Qwen Image Edit 2509
Running
on
CPU Upgrade
Featured
2.43k
The Smol Training Playbook
📚
2.43k
The secrets to building world-class LLMs
Runnin

In [14]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ntencent/HunyuanVideo-1.5\nUpdated\n1 day ago\n•\n1.68k\n•\n680\nfacebook/sam3\nUpdated\n5 days ago\n•\n115k\n•\n668\nblack-forest-labs/FLUX.2-dev\nUpdated\nabout 8 hours ago\n•\n59\n•\n396\nSupertone/supertonic\nUpdated\n4 days ago\n•\n4.11k\n•\n279\nfacebook/sam-3d-objects\nUpdated\n5 days ago\n•\n188\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\nMCP\nFeatured\n1.28k\nQwen Image Edit Came

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 7 relevant links


# Hugging Face - The AI Community Building the Future

---

## About Hugging Face

Hugging Face is a global collaboration platform dedicated to the machine learning (ML) community. It serves as a central hub where ML engineers, researchers, scientists, and enthusiasts can share, explore, and collaboratively develop open-source machine learning models, datasets, and applications.

Focused on openness and ethical AI development, Hugging Face empowers the next generation of AI professionals to learn, innovate, and build cutting-edge ML solutions in an inclusive environment.

---

## What Hugging Face Offers

- **Models**: Access and contribute to over 1 million machine learning models spanning text, image, video, audio, and 3D modalities. Popular models include those from industry leaders like Facebook and Tencent.
- **Datasets**: Browse a massive catalog of 250,000+ datasets enabling research and development across multiple domains such as autonomous vehicles, speech recognition, and synthetic data generation.
- **Spaces**: Deploy and share AI-powered applications easily with thousands of community-run apps covering various domains and use cases.
- **Community**: Engage with a thriving community of collaborators and contributors to share knowledge, best practices, and jointly advance ML technologies.
- **Enterprise Solutions**: Scalable paid compute and enterprise-grade tools to accelerate machine learning workflows for businesses.
- **Docs and Learning Resources**: Extensive documentation and guides helping users to build, deploy, and manage machine learning projects efficiently.

---

## Company Culture

Hugging Face fosters an open, collaborative, and community-driven culture emphasizing:

- **Ethical AI**: Promoting transparency and inclusivity in AI development to ensure responsible innovation.
- **Open Source**: Commitment to open-source technology as a foundation for accessible and democratized AI progress.
- **Community Empowerment**: Encouraging contributions from diverse users worldwide and supporting learning and growth opportunities.
- **Innovation at Scale**: Providing robust infrastructure and tools to enable rapid experimentation and deployment of ML models and applications.

---

## Who Uses Hugging Face?

Hugging Face serves a broad spectrum of users including:

- **Machine Learning Researchers and Engineers**: Exploring and sharing models and datasets to accelerate AI development.
- **Enterprises and Tech Companies**: Integrating cutting-edge AI capabilities into products and services.
- **AI Enthusiasts and Students**: Building skills and portfolios by contributing to open source ML projects.
- **Data Scientists and Developers**: Rapidly prototyping AI applications and integrating them into workflows.

---

## Careers at Hugging Face

Joining Hugging Face means becoming part of a passionate team dedicated to shaping the future of AI. The company values creativity, collaboration, and ethical responsibility.

- Positions are available in engineering, research, product management, and community engagement.
- Opportunities to work on state-of-the-art ML infrastructure and tools.
- A culture that supports continuous learning, diversity, and innovation.

For current job openings and application details, visit the company’s career page.

---

## Join the Future of AI

Whether you are building innovative AI applications, researching new models, or seeking a passionate community, Hugging Face is your platform to collaborate, learn, and scale your machine learning projects.

**Explore more and sign up:** [huggingface.co](https://huggingface.co)

---

*Empowering the AI community to build the future together.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is the AI community building the future of machine learning. It serves as a collaboration platform where the global machine learning community can share, explore, and develop models, datasets, and applications. Its aim is to empower machine learning engineers, scientists, and users worldwide to foster an open, ethical AI future.

---

## What We Offer

- **Hugging Face Hub:**  
  A central place to host, share, and collaborate on over 1 million public machine learning models and more than 250,000 datasets — across modalities including text, image, video, audio, and even 3D.  
- **Spaces:**  
  Run and share AI applications built by the community, facilitating seamless demonstration, experimentation, and collaboration.  
- **Open Source Stack:**  
  Access to the fast-growing suite of open-source ML tools and frameworks to accelerate innovation and development speed.  
- **Enterprise & Compute Services:**  
  Paid compute resources and enterprise-grade solutions to power your AI workloads efficiently and securely.

---

## Community & Culture

Hugging Face nurtures a vibrant, inclusive AI community centered on openness and collaboration. Members can build their machine learning portfolio, share their work globally, and learn from others across research and industry. The company actively supports ethical AI and open innovation as keys to progressing the future of artificial intelligence.

---

## Discover & Collaborate

- Explore trending models such as advanced video generation, segmentation, and image editing tools.  
- Dive into rich datasets curated for autonomous vehicles, multilingual speech recognition, synthetic data, and more.  
- Engage with an ecosystem of over 400,000 AI applications shared by creators worldwide.

---

## Career Opportunities

Joining Hugging Face means becoming part of a mission-driven organization committed to open science and cutting-edge AI. The company values innovation, collaboration, and expertise in machine learning, software development, data science, and AI research.

By working at Hugging Face, you will:  
- Contribute to widely-used open-source projects and shape the AI landscape.  
- Collaborate with a passionate, skilled team pushing the boundaries of ML technology.  
- Grow your career in an inclusive, fast-paced environment with global impact.

---

## Get Started

- **Join the community** by signing up and start exploring or sharing.  
- **Accelerate your machine learning workflows** with powerful tools and compute options.  
- **Build your AI portfolio** and gain recognition through public contributions.

Explore Hugging Face today —  
The Home of Machine Learning.

**Website:** https://huggingface.co

---

## Brand Colors & Logos

- Primary yellow: #FFD21E  
- Accent orange: #FF9D00  
- Neutral gray: #6B7280  

Logos and brand assets are available to promote consistent identity.

---

**Hugging Face — Enabling the machine learning community to innovate and build the future together.**

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 5 relevant links


# Hugging Face: Building the Future of AI

---

## Who We Are

Hugging Face is the premier **collaboration platform for the machine learning community**, dedicated to advancing open and ethical AI. We empower machine learning engineers, scientists, and end users around the globe to **create, discover, and collaborate** on models, datasets, and AI applications. Through our open-source ecosystem and thriving community, we stand at the heart of the AI revolution.

---

## Our Platform

- **Models Hub:** Access and share over **1 million pre-trained models** spanning text, image, video, audio, and even 3D modalities.
- **Datasets Library:** Discover and contribute to a diverse repository of **more than 250,000 datasets** frequently updated by the global community.
- **Spaces:** Host and explore thousands of ML-powered web applications built by developers worldwide.
- **Community:** Grow your portfolio, interact, and collaborate with thousands of ML practitioners.
- **Open Source Tools:** Leverage our industry-leading open-source ML libraries and tools to accelerate your AI projects.

---

## Enterprise Solutions

Hugging Face offers tailored solutions for businesses through our **Team and Enterprise Hubs** with features including:

- Enterprise-grade security with **Single Sign-On (SSO)** and granular access controls.
- Comprehensive **audit logs** and **resource groups** for streamlined governance.
- Scalable infrastructure with **advanced compute options** like ZeroGPU and quota boosts.
- Enhanced collaboration via **private storage**, **private dataset viewers**, and centralized token management.
- Usage analytics and spending controls for efficient resource management.
- Flexible subscription plans starting at $20/user/month for teams, with custom contracts for enterprises.

---

## Our Culture & Community

At Hugging Face, we nurture a **culture of openness, innovation, and collaboration**. We believe in building an ethical AI future by bringing people together — from budding ML students to experienced researchers and industry professionals. Our fast-growing community contributes to some of the most widely used open-source ML projects worldwide, fostering an environment that encourages learning and sharing.

---

## Careers

Join us to be part of a talented and passionate science and engineering team exploring the cutting edge of AI technology. At Hugging Face, you will have the opportunity to:

- Work on impactful open-source projects used by millions.
- Collaborate with a vibrant and supportive global community.
- Grow your skills and career while contributing to the AI ecosystem.
- Engage in a mission-driven company committed to transparency and ethical AI.

Check our [Jobs page](https://huggingface.co/jobs) to explore current opportunities.

---

## Why Choose Hugging Face?

- **Vibrant ecosystem:** Millions of models and datasets ready to use and contribute to.
- **Community-driven:** A global network of AI enthusiasts and professionals collaborating daily.
- **Robust infrastructure:** Scalable, secure, and enterprise-friendly platform support.
- **Commitment to ethics:** Open and transparent development fostering responsible AI innovation.

---

## Connect With Us

- Explore models, datasets, and apps at [huggingface.co](https://huggingface.co)
- Join our discussions on [Discord](https://discord.gg/huggingface), [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), and [LinkedIn](https://linkedin.com/company/huggingface)
- Learn through our Documentation, Blog, and Forums

---

**Hugging Face — The AI community building the future.**  
Unlock the power of machine learning, together.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>